# Codelab - GenAI - Image Generation from keywords

A copy of the notebook / code from the Codelab: [GenAI - Image Generation from keywords](https://codelabs.developers.google.com/image-generation-using-keywords#0)

<h1> Install All Dependencies </h1>

In [ ]:
#Install All Required Modules and latest vertex ai version
# !pip install gradio
# !pip install google-cloud-aiplatform --upgrade

<h1> Image Generation Using Imagen </h1>

In [ ]:
from vertexai.preview.vision_models import ImageGenerationModel
from PIL import Image
import base64 
from io import BytesIO
import gradio as gr

# Function to Generate Images from the Input Prompt and Negative Prompt (Optional)
def image_generation_completion(input, negative_prompt):  
    input_prompt = input
    model = ImageGenerationModel.from_pretrained("imagegeneration@002")
    response = model.generate_images(
        prompt=input_prompt,
        number_of_images=4, #kept to static value of 4
        negative_prompt=negative_prompt
    )
    images = response.images  
    image_return_list = []
    for i in range(4):
        image_return_list.append(gr.update(
            value = images[i]._pil_image,  visible = True))
    #Returns Generated Images
    return image_return_list

<h1> Prompt Generation Using Text-Bison </h1>

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel
def prompt_generation(persona,signal,theme, lighting, quality, extra_desc):
    params_list = [persona,signal,theme, lighting, quality, extra_desc]
    params_list_str = ""
    for param in params_list:
        if len(param) > 0 and param is not None:
            params_list_str = params_list_str + param + ", "
    params_list_str = params_list_str[:-2]
    
    vertexai.init(
        # project="my-project-1-361607", 
                  location="us-central1")
    parameters = {
        "max_output_tokens": 256,
        "temperature": 0.2,
        "top_p": 0.8,
        "top_k": 40
    }
    few_shot_prompt = f"""You are an expert in writing prompts for Image Generation Models. Using the provided phrases and keywords, concatenate them and add on some realistic details to generate logical and Meaningful prompt that can be used for image generation.

    input: people, gardening, house garden, colorful plants, Real, HD image, Photo.
    output: A Photo of people gardening in a house garden landscape with few coloured flowering plants. Realistic FULL HD Images, Elegant and natural facial and eye features taken by professional photographer

    input: plumber, faucet, kitchen, high quality, natural lighting, Photo
    output: A Photo of a plumber fixing a faucet in the kitchen. High quality image with natural indoor lighting.

    input: house and garden, halloween, warm lighting, high quality image, Sketch
    output: A Sketch of Beautiful House and Garden with Halloween Decorations. Warm lighting, High Quality, 4K photograph taken by professional photographer from front.

    input: nice living room, warm lighting,Professional Photographer from far, Photo
    output: A photo of a Well designed Living Room. Warm lighting, High Quality, 4K photograph taken by Professional Photographer from far

    input: {params_list_str}
    output:
    """
    
    prompt = f"""You are an expert in writing prompts for Image Generation Models. Help me write a list of meaningful prompts for Image Generation Model specifically including the words: "{params_list_str}". Remember to include these words in the prompt and make the prompt meaningful."""
    model = TextGenerationModel.from_pretrained("text-bison")
    response_few_shot = model.predict(
        few_shot_prompt,
        **parameters
    )
    response_single_shot = model.predict(
        prompt,
        **parameters
    )
    
    output_prompt = "Result Prompt: \n" + response_few_shot.text + "\n\n" + "Similar Prompts: \n" + response_single_shot.text
    return output_prompt


<h1> Gradio Application - Consolidation of Prompts and Images Generation via Gradio UI</h1>

In [ ]:
import gradio as gr


def populate_image_prompt(prompt, selectEvent: gr.SelectData):
    return gr.update(value = selectEvent.value)
    

gr.close_all()
with gr.Blocks() as demo:
    
    #Prompt Generation Part
    with gr.Row():
        with gr.Column(scale=1):
            Persona = gr.Textbox(label="Persona", info = "Customer segment such as Plumber, Electrician etc.")
        with gr.Column(scale=1):
            Signals = gr.Textbox(label="Signals", info = "Main content of banner such as Faucet, Lamp etc.")
        with gr.Column(scale=1):
            Theme = gr.Textbox(label="Theme", info = "Context of the banner such as Halloween, Kitchen etc.")
    with gr.Row():
        with gr.Column(scale=1):
            photo_modifiers = gr.Textbox(label="Photography Modifiers", info = "Photography specific modifiers and parameters such as Lighting(Dramatic/Natural/Warm/Cold), Camera Proximity etc.")
        with gr.Column(scale=1):
            quality_modifiers =  gr.Textbox(label="Image Quality Modifier", info = "Quality Modifiers like high-quality, beautiful, stylized. 4K, HDR, By a professional etc")
        with gr.Column(scale=1):
            other_desc =  gr.Textbox(label="Any Other Description", info = "Other Descriptions for Image such as Style (Painting/Photo/Sketch), Bakground/Foreground Context")
                
    with gr.Row():
        btn = gr.Button("Submit")
    with gr.Row():
        returned_prompts = gr.Textbox(label="Result Prompts", interactive = True)    
    btn.click(fn=prompt_generation, inputs=[Persona, Signals,Theme, photo_modifiers, quality_modifiers, other_desc], outputs = returned_prompts)
    
    
    
    #Image Generation part
    with gr.Row():
        with gr.Column(scale=1):
            image_prompt = gr.Textbox(label="Image Generation Prompt")
    with gr.Accordion("Advanced options", open=False): #Let's hide the advanced options!
        with gr.Row():
            negative_prompt = gr.Textbox(label="Negative prompt", info = "Specify What not to Include in Image ex. Bad Quality Image")
    with gr.Row():
        with gr.Column(scale=1):    
            img_btn = gr.Button("Generate Images")
    with gr.Row():
        with gr.Column():
            output_image_1 = gr.Image(label = "Result Image 1", visible = False)
        with gr.Column():
            output_image_2 = gr.Image(label = "Result Image 2", visible = False)
    with gr.Row():
        with gr.Column():
            output_image_3 = gr.Image(label = "Result Image 3", visible = False)
        with gr.Column():
            output_image_4 = gr.Image(label = "Result Image 4", visible = False)
            
    returned_prompts.select(populate_image_prompt, inputs = [returned_prompts], outputs = image_prompt)

    img_btn.click(fn=image_generation_completion, inputs=[image_prompt,negative_prompt], outputs = [output_image_1,output_image_2,output_image_3,output_image_4])       

    
gr.close_all()
demo.close()
demo.launch(share=False)

<h1> Image Generation in Notebook Using in-code Prompts </h1>

In [ ]:
def image_generation(input, negative_prompt):  
    input_prompt = input
    model = ImageGenerationModel.from_pretrained("imagegeneration@002")
    response = model.generate_images(
        prompt=input_prompt,
        number_of_images=4, #kept to static value of 4.. can be a max value of 8
        negative_prompt=negative_prompt
    )
    images = response.images
    return images


user_prompt = "A Candid photo of a Young Professional Gardener wearing a green Gardener Uniform mowing a big lawn with plants \
               and flowers outside a house. Natural Lighting, High Quality, Natural and Elegant Facial Features Photo."
negative_prompt = "Distorted and unattractive faces"
generated_images_list = image_generation(user_prompt,negative_prompt)

In [ ]:
#show one of the generated image
generated_images_list[1].show()